In [1]:
import os
import pandas as pd
import numpy as np
import awkward0
import uproot3_methods

In [2]:
import logging
logging.basicConfig(level=logging.DEBUG, format='[%(asctime)s] %(levelname)s: %(message)s')

In [9]:
cd ..

/home/rd804/discriminator-metric


In [10]:
pwd

'/home/rd804/discriminator-metric'

In [5]:
def _transform(data, label, jet_size=0.8):
    from collections import OrderedDict
    v = OrderedDict()

    _e = data[...,0]
    _px = data[...,1]
    _py = data[...,2]
    _pz = data[...,3]
    
    mask = _e>0
    n_particles = np.sum(mask, axis=1)

   # print('num particles: ',n_particles[0:20])

    px = awkward0.JaggedArray.fromcounts(n_particles, _px[mask])
    py = awkward0.JaggedArray.fromcounts(n_particles, _py[mask])
    pz = awkward0.JaggedArray.fromcounts(n_particles, _pz[mask])
    energy = awkward0.JaggedArray.fromcounts(n_particles, _e[mask])

    p4 = uproot3_methods.TLorentzVectorArray.from_cartesian(px, py, pz, energy)

    #print(p4)
    pt = p4.pt

    jet_p4 = p4.sum()

    # outputs
    _label = label
    v['label'] = np.stack((_label, 1-_label), axis=-1)
    #v['train_val_test'] = df['ttv'].values
    
    v['jet_pt'] = jet_p4.pt
    v['jet_eta'] = jet_p4.eta
    v['jet_phi'] = jet_p4.phi
    v['jet_mass'] = jet_p4.mass
    v['n_parts'] = n_particles

    v['part_px'] = px
    v['part_py'] = py
    v['part_pz'] = pz
    v['part_energy'] = energy

    v['part_pt_log'] = np.log(pt)
    v['part_ptrel'] = pt/v['jet_pt']
    v['part_logptrel'] = np.log(v['part_ptrel'])

    v['part_e_log'] = np.log(energy)
    v['part_erel'] = energy/jet_p4.energy
    v['part_logerel'] = np.log(v['part_erel'])

    v['part_raw_etarel'] = (p4.eta - v['jet_eta'])
    _jet_etasign = np.sign(v['jet_eta'])
    _jet_etasign[_jet_etasign==0] = 1
    v['part_etarel'] = v['part_raw_etarel'] * _jet_etasign

    v['part_phirel'] = p4.delta_phi(jet_p4)
    v['part_deltaR'] = np.hypot(v['part_etarel'], v['part_phirel'])

    def _make_image(var_img, rec, n_pixels = 64, img_ranges = [[-0.8, 0.8], [-0.8, 0.8]]):
        wgt = rec[var_img]
        x = rec['part_etarel']
        y = rec['part_phirel']
        img = np.zeros(shape=(len(wgt), n_pixels, n_pixels))
        for i in range(len(wgt)):
            hist2d, xedges, yedges = np.histogram2d(x[i], y[i], bins=[n_pixels, n_pixels], range=img_ranges, weights=wgt[i])
            img[i] = hist2d
        return img

#     v['img'] = _make_image('part_ptrel', v)

    return v

In [6]:
def convert(data,label, destdir, basename, step=None, limit=None):

    if not os.path.exists(destdir):
        os.makedirs(destdir)
    output = os.path.join(destdir, '%s.awkd'%(basename))
    logging.info(output)

    if os.path.exists(output):
        logging.warning('... file already exist: continue ...')
        return
    v=_transform(data,label)
    awkward0.save(output, v, mode='x')

In [7]:
srcDir = 'data'
destDir = 'data/converted'

In [11]:
# convert training file
test_jets = ['shifted','eta_smeared','smeared','pt_shifted','pt_smeared','all_smeared']
split = ['train', 'valid']

for test_jet in test_jets:
    for s in split:
        particle_data = pd.read_hdf('data/jetnet30_data.h5', f'particle_data_{test_jet}_{s}').values.reshape(-1,30,4)
        labels = pd.read_hdf('data/jetnet30_data.h5', f'labels__{test_jet}_{s}').values
        convert(data=particle_data,label=labels, destdir=destDir, basename=f'{s}_{test_jet}_file')

[2023-04-17 18:55:41,912] INFO: data/converted/train_shifted_file.awkd
[2023-04-17 18:55:43,781] INFO: data/converted/valid_shifted_file.awkd
[2023-04-17 18:55:44,752] INFO: data/converted/train_eta_smeared_file.awkd
[2023-04-17 18:55:47,906] INFO: data/converted/valid_eta_smeared_file.awkd
[2023-04-17 18:55:49,878] INFO: data/converted/train_smeared_file.awkd
[2023-04-17 18:55:52,300] INFO: data/converted/valid_smeared_file.awkd
[2023-04-17 18:55:55,852] INFO: data/converted/train_pt_shifted_file.awkd
[2023-04-17 18:55:59,936] INFO: data/converted/valid_pt_shifted_file.awkd
[2023-04-17 18:56:04,070] INFO: data/converted/train_pt_smeared_file.awkd
[2023-04-17 18:56:10,066] INFO: data/converted/valid_pt_smeared_file.awkd
[2023-04-17 18:56:18,112] INFO: data/converted/train_all_smeared_file.awkd
[2023-04-17 18:56:21,839] INFO: data/converted/valid_all_smeared_file.awkd


In [10]:
# convert validation file


In [11]:
# convert testing file
